In [11]:

import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns

In [12]:
!make clean && make all

rm -rf *.o
rm main
g++ -o main main.cpp solucion.cpp -O2 -std=c++14


In [13]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [14]:
def save_instance(dataset, instance_name, N, impacto, tiempo):
    with open(F"resultados/{dataset}/{instance_name}.txt", "w") as f:
        lista = [N,impacto,tiempo]
        for i in range(len(lista)):
            print(lista[i], file=f, sep=' ', end =' ')

In [6]:
save_instance("Experimento1", "a", 1,0,1);
save_instance("Experimento1", "a", 2,0,1);

In [7]:
CANT_REPETIONES_EXPERIMENTOS = 20

In [8]:
def correr_experimento(metodo, archivo_instancia, cantNodos):
    
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["./main", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)
    
    
    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.write("\n")
    process.stdin.flush()
   
    #print(instancia)
    
    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    resultados = []
    tiempo = float(process.stderr.read())
    resultados.append(tiempo)
    
    resultado = process.stdout.read();    
    resultado = resultado.split()
    resultado = map(int, resultado)
    resultado = list(resultado)
  
    resultados += resultado 
    
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return resultados;

In [9]:
archivos= []
run = []
for i in range(31):
    if ( (i > 5) & (i % 2 == 0)):
        archivos.append(f"instancias/CMI_n{i}.in")
        run.append(i)
            

In [9]:

for i in range(len(archivos)):
    print(correr_experimento("M2", archivos[i],run[i]))

[441.256, 2, 2, 0, 1, 0, 2, 0]
[476.071, 6, 0, 1, 1, 0, 0, 0, 1, 2]
[1183.35, 4, 1, 0, 1, 1, 0, 3, 0, 3, 2, 1]
[784.624, 16, 0, 0, 0, 2, 1, 1, 1, 2, 0, 0, 0, 1]
[1492.4, 12, 2, 0, 0, 2, 3, 1, 0, 1, 0, 2, 3, 1, 2, 0]
[1826.84, 19, 3, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 3, 1, 1, 3, 1]
[7344.03, 24, 3, 1, 2, 0, 2, 2, 3, 2, 0, 1, 1, 0, 1, 0, 0, 0, 0, 2]
[2860.12, 20, 2, 3, 3, 4, 1, 0, 0, 2, 4, 0, 2, 1, 4, 5, 1, 0, 2, 2, 3, 1]
[4453.56, 26, 0, 0, 0, 5, 4, 1, 3, 0, 3, 1, 0, 0, 4, 0, 0, 0, 3, 2, 1, 2, 4, 1]
[5921.13, 27, 4, 4, 1, 1, 2, 0, 0, 4, 2, 0, 0, 1, 3, 3, 2, 0, 2, 3, 1, 3, 0, 1, 2, 5]
[7942.67, 37, 2, 1, 4, 3, 3, 0, 1, 3, 1, 0, 2, 1, 3, 2, 2, 0, 0, 1, 2, 3, 0, 4, 0, 0, 1, 2]
[7805.11, 44, 3, 1, 3, 1, 4, 0, 2, 0, 0, 1, 4, 2, 2, 1, 1, 0, 1, 4, 0, 2, 4, 1, 2, 0, 3, 0, 0, 3]
[12762.9, 46, 0, 4, 2, 3, 3, 0, 2, 4, 4, 2, 3, 0, 0, 3, 2, 0, 0, 1, 0, 3, 2, 2, 2, 0, 1, 5, 3, 3, 1, 1]


# parametros

In [10]:
parametros = ["N", "Impacto", "Tiempo"]


# Experimento 1: Comparacion de heuristicas 


## Heuristica 1

In [10]:

parametros = ["N", "Impacto", "Tiempo"]
filas = []
for i in range((len(archivos))): 
    promedio = 0
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("H1", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento1/h1.csv", index=False, header=True)           
    


## Heuristica 2

In [11]:

filas = []
for i in range((len(archivos))): 
    promedio = 0
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("H2", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento1/h2.csv", index=False, header=True)           
    


## Aca se crea el csv de los resultados de la catedra

In [20]:
parametros_sin_tiempo = ["N", "Impacto"]
archivos_catedra= []
run = []
filas = []
for i in range(31):
    if ( (i > 5) & (i % 2 == 0)):
        with open(f"instancias/CMI_n{i}.out") as f:
            c = f.read(2)
            print(c)
            lista = [i,c]
            filas.append(lista)

pd.DataFrame(filas, columns=parametros_sin_tiempo).to_csv("resultados/best_results.csv", index=False, header=True)   

1 
6 
3 
16
12
20
27
25
26
33
38
48
47


# Experimento 3 

## Metaheuristica M2

### Corrido con H2

In [31]:
filas = []
cant_experimentos = len(archivos)*CANT_REPETIONES_EXPERIMENTOS
cant_terminados = 0
barra= "/" 
for i in range((len(archivos))): 
    promedio = 0
    promedio_impacto=0 
    
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("M2", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        promedio_impacto+= lista[1]
        cant_terminados+=1
        clear_output(wait=True)
        display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
    
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    lista[2] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento3/M2H2.csv", index=False, header=True)           


'Experimento: 260/260'

### Corrido con H1

In [11]:
filas = []
cant_experimentos = len(archivos)*CANT_REPETIONES_EXPERIMENTOS
cant_terminados = 0
barra= "/" 
for i in range((len(archivos))): 
    promedio = 0
    promedio_impacto=0 
    
    for j in range(CANT_REPETIONES_EXPERIMENTOS):
        prueba_resultados = correr_experimento("M2", archivos[i],run[i])
        lista = [(len(prueba_resultados) -2),prueba_resultados[1], prueba_resultados[0]]
        promedio+= lista[2]
        promedio_impacto+= lista[1]
        cant_terminados+=1
        clear_output(wait=True)
        display('Experimento: ' + str(cant_terminados) + "/" + str(cant_experimentos))
    
    lista[2] = promedio/CANT_REPETIONES_EXPERIMENTOS
    lista[2] = promedio_impacto/CANT_REPETIONES_EXPERIMENTOS
    filas.append(lista)
    pd.DataFrame(filas, columns=parametros).to_csv("resultados/Experimento3/M2H1.csv", index=False, header=True)           


'Experimento: 260/260'